In [ ]:
import pandas as pd
from faker import Faker
import random

def generate_tables(num_rows):
    faker = Faker()
    
    # Generate data for Product table
    product_data = {
        'product_id': list(range(1, num_rows + 1)),
        'product_name': [faker.word() for _ in range(num_rows)],
        'unit_price': [random.randint(10, 100) for _ in range(num_rows)]
    }
    product_df = pd.DataFrame(product_data)
    
    # Save Product table to CSV
    product_df.to_csv('product_table.csv', index=False)
    
    # Generate data for Sales table
    sales_data = {
        'seller_id': [random.randint(100, 200) for _ in range(num_rows)],
        'product_id': [random.randint(1, num_rows) for _ in range(num_rows)],
        'buyer_id': [random.randint(300, 400) for _ in range(num_rows)],
        'sale_date': [faker.date_between(start_date='-1y', end_date='today') for _ in range(num_rows)],
        'quantity': [random.randint(1, 5) for _ in range(num_rows)],
        'price': [random.randint(10, 100) for _ in range(num_rows)]
    }
    sales_df = pd.DataFrame(sales_data)
    
    # Save Sales table to CSV
    sales_df.to_csv('sales_table.csv', index=False)

# Example usage: Generate tables with 10 rows each
generate_tables(100)


In [ ]:
df1=pd.read_csv('product_table.csv')
#iterate over the rows of the dataframe
for index, row in df1.iterrows():
    print(row['product_id'], row['product_name'], row['unit_price'])

In [8]:
import pandas as pd


def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    temp_res={
    }
    for i,row in sales.iterrows():
        # print(row)
        try:
            temp_res[row.buyer_id].add(row.product_id)
        except:
            
            if row.buyer_id not in temp_res:
                temp_res[row.buyer_id]=set(row.product_id)
    print(temp_res)
    
    
    
    return sales


In [23]:
import pandas as pd


def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    temp_res = {}
    merged=pd.merge(product,sales,on='product_id')
    for i, row in merged.iterrows():
        if row.buyer_id not in temp_res:
            temp_res[row.buyer_id] = set()
            temp_res[row.buyer_id].add(row.product_name)
        else:
            temp_res[row.buyer_id].add(row.product_name)
    # print(temp_res)
    res=[]
    for i in temp_res:
        if 'S8' in temp_res[i] and 'iPhone' not in temp_res[i]:
            res.append(i)
    
    res=pd.DataFrame({'buyer_id':res})
    return res
sales_analysis(pd.read_csv('product_table.csv'),pd.read_csv('sales_table.csv'))


,buyer_id


In [30]:
import pandas as pd


merged=pd.merge(pd.read_csv('product_table.csv'),pd.read_csv('sales_table.csv'),on='product_id')
s8=merged[merged['product_name']=='S8']
iPhone=merged[merged['product_name']=='iPhone']
res=s8[~s8['buyer_id'].isin(iPhone['buyer_id'])]
res=res.drop_duplicates(subset=['buyer_id'])
return res

,product_id,product_name,unit_price,seller_id,buyer_id,sale_date,quantity,price


In [31]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime

# Function to generate random dates within a range
def random_dates(start, end, n=10):
    start_u = start.value//10**9
    end_u = end.value//10**9
    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='s')

# Define the start and end date for random date generation
start = pd.to_datetime('2019-01-01')
end = pd.to_datetime('2019-12-31')

# Generate the Product DataFrame
product_df = pd.DataFrame({
    'product_id': range(1, 101),
    'product_name': np.random.choice(['S8', 'G4', 'iPhone', 'Note', 'Pixel'], 100),
    'unit_price': np.random.randint(300, 1500, size=100)
})

# Generate the Sales DataFrame
sales_df = pd.DataFrame({
    'seller_id': np.random.randint(1, 10, size=100),
    'product_id': np.random.randint(1, 101, size=100),
    'buyer_id': np.random.randint(1, 50, size=100),
    'sale_date': random_dates(start, end, 100),
    'quantity': np.random.randint(1, 5, size=100),
    'price': np.random.randint(300, 1500, size=100) * np.random.randint(1, 5, size=100)
})

print("Product Table:")
print(product_df.head())  # Show only the first 5 entries for brevity
print("\nSales Table:")
print(sales_df.head())  # Show only the first 5 entries for brevity


Product Table:
   product_id product_name  unit_price
0           1           S8        1244
1           2           S8        1033
2           3         Note         323
3           4        Pixel        1271
4           5         Note         306

Sales Table:
   seller_id  product_id  buyer_id           sale_date  quantity  price
0          4          46        46 2019-10-26 05:08:21         3   2610
1          6           1        13 2019-07-22 22:45:17         1   1326
2          1           7        37 2019-01-01 21:25:32         1   4724
3          7          20        23 2019-07-10 07:04:05         1   1488
4          8          89        19 2019-01-13 11:42:30         3   2259


In [46]:
merged=pd.merge(product_df,sales_df,on='product_id')
first_q=merged[merged['sale_date']<='2019-03-31']
other_qs=merged[merged['sale_date']>'2019-03-31']
res=first_q[~first_q.product_id.isin(other_qs.product_id)]
res=res.drop_duplicates()
res[['product_id','product_name']]


,product_id,product_name
3,5,Note
4,7,S8
5,8,G4
6,8,G4
7,9,G4
15,14,Note
28,27,Pixel
46,42,S8
47,43,Pixel
59,59,S8


In [67]:
def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    product_id={}
    first_q=set()
    other_qs=set()
    for i,row in sales.iterrows():
        if pd.to_datetime('2019-03-31') >= row['sale_date'] >= pd.to_datetime('2019-01-01'):
            first_q.add(row.product_id)
        if pd.to_datetime('2019-03-31') < row['sale_date'] < pd.to_datetime('2019-12-31'):
            other_qs.add(row.product_id)
    
            
    # print(first_q-other_qs)
    res_id=list(first_q-other_qs)
    print(product_id)
    return product_id


sales_analysis(product_df, sales_df)

{'product_id': {0: 1, 2: 3, 3: 4, 5: 6, 49: 50}, 'product_name': {0: 'S8', 2: 'Note', 3: 'Pixel', 5: 'G4', 49: 'iPhone'}}


{'product_id': {0: 1, 2: 3, 3: 4, 5: 6, 49: 50},
 'product_name': {0: 'S8', 2: 'Note', 3: 'Pixel', 5: 'G4', 49: 'iPhone'}}